In [1]:
import pandas as pd
import os
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
pd.set_option("display.max_columns", 100)

In [2]:
### file directory for mimic ###
hosp_path ="./mimic-iv-2.2/hosp/"
icu_path ="./mimic-iv-2.2/icu/"
hosp_all_files = os.listdir(hosp_path)
icu_all_files = os.listdir(icu_path)

In [3]:
ipe = pd.read_csv(icu_path+'inputevents.csv.gz')
ipe.head()

,subject_id,hadm_id,stay_id,caregiver_id,starttime,endtime,storetime,itemid,amount,amountuom,rate,rateuom,orderid,linkorderid,ordercategoryname,secondaryordercategoryname,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,totalamountuom,isopenbag,continueinnextdept,statusdescription,originalamount,originalrate
0,10000032,29079034,39553978,66056,2180-07-23 21:10:00,2180-07-23 21:11:00,2180-07-23 21:10:00,226452,100.000000,ml,NaN,NaN,415918,415918,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.0
1,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:01:00,2180-07-23 18:56:00,226452,200.000000,ml,NaN,NaN,7140773,7140773,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,39.4,200.0,ml,0,0,FinishedRunning,200.0,200.0
2,10000032,29079034,39553978,88981,2180-07-23 17:00:00,2180-07-23 17:30:00,2180-07-23 17:02:00,220862,49.999999,ml,100.0,mL/hour,7578214,7578214,04-Fluids (Colloids),NaN,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.0
3,10000032,29079034,39553978,88981,2180-07-23 17:33:00,2180-07-23 18:03:00,2180-07-23 18:16:00,220862,49.999999,ml,100.0,mL/hour,427849,427849,04-Fluids (Colloids),NaN,Main order parameter,Continuous IV,39.4,50.0,ml,0,0,FinishedRunning,50.0,100.0
4,10000032,29079034,39553978,88981,2180-07-23 18:56:00,2180-07-23 18:57:00,2180-07-23 18:56:00,226452,100.000000,ml,NaN,NaN,1579487,1579487,14-Oral/Gastric Intake,NaN,Main order parameter,Bolus,39.4,100.0,ml,0,0,FinishedRunning,100.0,100.0


In [4]:
ditems =pd.read_csv(icu_path+'d_items.csv.gz')
ditems.head(300)

,itemid,label,abbreviation,linksto,category,unitname,param_type,lownormalvalue,highnormalvalue
0,220001,Problem List,Problem List,chartevents,General,NaN,Text,NaN,NaN
1,220003,ICU Admission date,ICU Admission date,datetimeevents,ADT,NaN,Date and time,NaN,NaN
2,220045,Heart Rate,HR,chartevents,Routine Vital Signs,bpm,Numeric,NaN,NaN
3,220046,Heart rate Alarm - High,HR Alarm - High,chartevents,Alarms,bpm,Numeric,NaN,NaN
4,220047,Heart Rate Alarm - Low,HR Alarm - Low,chartevents,Alarms,bpm,Numeric,NaN,NaN
...,...,...,...,...,...,...,...,...,...
295,221689,Drotrecogin (Xigris),Drotrecogin (Xigris),inputevents,Medications,mg,Solution,NaN,NaN
296,221712,Ketamine,Ketamine,inputevents,Medications,mg,Solution,NaN,NaN
297,221733,Factor VIII,Factor VIII,inputevents,Blood Products/Colloids,International Units,Solution,NaN,NaN
298,221744,Fentanyl,Fentanyl,inputevents,Medications,mg,Solution,NaN,NaN


In [5]:
#merge itemlist with input to get label
ditems_f = ditems[ditems['linksto']=='inputevents']
final = pd.merge(left=ipe, right=ditems_f, on='itemid', how='left')
#find out the subject id who received enoxaparin and heparin 


In [6]:
for i in ditems_f['label'].unique():
    if ('Hepa' in i or ('hepa' in i) or ('Enox' in i) or ('enox' in i)):
        print(i)

Heparin Sodium
Enoxaparin (Lovenox)
Hepatamine
Heparin Sodium (Prophylaxis)
Heparin Sodium (Impella)
Heparin Sodium (CRRT-Prefilter)


In [7]:
## filter only for specific label
A_B = ['Enoxaparin (Lovenox)', 'Heparin Sodium (Prophylaxis)']
temp = final[final['label'].isin(A_B)]
temp.head()
temp['ordercategoryname'].describe()

count                      154294
unique                          1
top       11-Prophylaxis (Non IV)
freq                       154294
Name: ordercategoryname, dtype: object

In [8]:
len(temp[temp['label']==A_B[0]]), len(temp[temp['label']==A_B[1]])

(3489, 150805)

In [9]:
## remove duplicates per SHS + label
temp2 = temp[['subject_id', 'hadm_id', 'stay_id', 'label']].drop_duplicates()
temp3 = temp2.groupby(by=['subject_id','hadm_id','stay_id'])['label'].count().reset_index().rename(columns={'label': 'count'})
## count if SHS has taken both, we interested only if either or, so filter for count == 1
temp4 = pd.merge(left=temp2, right=temp3, on=['subject_id','hadm_id','stay_id'], how='left')
temp5 = temp4[temp4['count']==1]

In [10]:
#temp[(temp['subject_id']==10013569) & (temp['hadm_id']==27993048) & (temp['stay_id']==38857852)]
len(temp5)

24743

In [11]:
# filter for first care unit
fcu_filter = ["Surgical Intensive Care Unit (SICU)", "Medical/Surgical Intensive Care Unit (MICU/SICU)"]
icustay =pd.read_csv(icu_path+'icustays.csv.gz')
# unique subject_id filter 
icustay = icustay[icustay['first_careunit'].isin(fcu_filter)].sort_values(by='intime').groupby(by=['subject_id']).first().reset_index()
icustay.head(10)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032
1,10001725,25563031,31205490,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-04-11 15:52:22,2110-04-12 23:59:56,1.338588
2,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447
3,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222
4,10002930,25922998,35629889,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2198-04-17 21:24:00,2198-04-18 13:41:43,0.678970
5,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213
6,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308
7,10004720,22081550,35009126,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2186-11-12 19:55:00,2186-11-17 21:15:55,5.056192
8,10004733,27411876,39635619,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2174-12-04 11:28:24,2174-12-12 20:03:01,8.357373
9,10006131,27849136,30177745,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2143-03-30 00:27:00,2143-03-31 17:56:38,1.728912


In [12]:
icu_label_combine = pd.merge(left=icustay, right=temp5, on=['subject_id', 'hadm_id', 'stay_id'], how='left')
icu_label_combine = icu_label_combine.dropna(subset=['label'])
icu_label_combine.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0
2,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0
3,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0
5,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0
6,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0


In [13]:
f"{A_B[0]}: {len(icu_label_combine[icu_label_combine['label']==A_B[0]])}, {A_B[1]}: {len(icu_label_combine[icu_label_combine['label']==A_B[1]])}, total: {len(icu_label_combine)}"

'Enoxaparin (Lovenox): 316, Heparin Sodium (Prophylaxis): 7638, total: 7954'

In [14]:
admit =pd.read_csv(hosp_path+'admissions.csv.gz')
ptage =pd.read_csv(hosp_path+'patients.csv.gz')
admit_ptage_combine = pd.merge(left=admit, right=ptage, on='subject_id', how='left')
admit_ptage_combine['admittime'] = pd.to_datetime(admit_ptage_combine['admittime'])
admit_ptage_combine['real_age']= admit_ptage_combine.apply(lambda x: x['admittime'].year - x['anchor_year'] + x['anchor_age'], axis=1)
#the age of a patient = admission time - anchor_year + anchor_age, this is based on sql page

In [15]:
icu_label_admit_ptage_combine = pd.merge(left=icu_label_combine, right=admit_ptage_combine, on=['subject_id', 'hadm_id'], how='left')
icu_label_admit_ptage_combine = icu_label_admit_ptage_combine[icu_label_admit_ptage_combine['real_age']>=18]
icu_label_admit_ptage_combine.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75


In [16]:
len(set(icu_label_admit_ptage_combine['subject_id'])) # unique per subject id

7954

In [17]:
len(icu_label_admit_ptage_combine[icu_label_admit_ptage_combine['label']==A_B[0]]), len(icu_label_admit_ptage_combine[icu_label_admit_ptage_combine['label']==A_B[1]]), len(icu_label_admit_ptage_combine)

(316, 7638, 7954)

In [18]:
#read icd csv
icd_dx =pd.read_csv(hosp_path+'d_icd_diagnoses.csv.gz')
dx =pd.read_csv(hosp_path+'diagnoses_icd.csv.gz')
icd_dx_combine = pd.merge(left=icd_dx, right=dx, on=['icd_code','icd_version'], how='left')
icd_dx_combine = icd_dx_combine.dropna(subset=['subject_id'])
icd_dx_combine[['subject_id', 'hadm_id']] =icd_dx_combine[['subject_id', 'hadm_id']].astype(int)

In [19]:
combine1 = pd.merge(left=icu_label_admit_ptage_combine, right=icd_dx_combine, on=['subject_id', 'hadm_id'], how='left')

In [20]:
import numpy as np
def check_unique_subject_id(df):
    print(f"total_len: {len(df['subject_id'])}, total_unique_id: {len(set(df['subject_id']))}")
len(np.unique(combine1['subject_id']))

7954

In [21]:
sofa =pd.read_csv("./first_day_sofa.csv")
charlson =pd.read_csv("./charlson.csv").drop_duplicates()
avgplatlets =pd.read_csv("./avg_pl.csv")
age_eth_wt =pd.read_csv("./Age_ethnicity_weight.csv")
creatinine =pd.read_csv("./creat_score.csv")
mortality =pd.read_csv("./mortality.csv")
vasopr =pd.read_csv("./update_Concurrent_vasopressor_use.csv")
ventil = pd.read_csv("./ventilation.csv").drop_duplicates()
ventil = pd.merge(left=ventil, right=ventil.groupby('subject_id')['ventilation_st'].sum().reset_index().rename(columns={'ventilation_st': 'have_at_least_1_ventil'}), on='subject_id', how='left')
ventil = ventil[['subject_id','hadm_id','stay_id', 'have_at_least_1_ventil']].drop_duplicates()
ventil['have_at_least_1_ventil'] = ventil['have_at_least_1_ventil'].clip(upper=1)

In [22]:
print('Check for duplicated subject_id')
check_unique_subject_id(charlson)
check_unique_subject_id(avgplatlets)
check_unique_subject_id(age_eth_wt)
check_unique_subject_id(creatinine)
check_unique_subject_id(mortality)
check_unique_subject_id(vasopr)
check_unique_subject_id(ventil)

Check for duplicated subject_id
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954
total_len: 7954, total_unique_id: 7954


In [23]:

# check if charlson is repeated in subject_id
# charlson2 = pd.merge(left=charlson, right=charlson.groupby('subject_id')['hadm_id'].count().reset_index().rename(columns={'hadm_id': 'count'}), on='subject_id', how='left')
# print(len(charlson2[charlson2['count']>1].sort_values(by='subject_id')), len(charlson2[charlson2['count']>1].sort_values(by='subject_id').drop_duplicates()))
# charlson2[charlson2['count']>1].sort_values(by='subject_id')

# we only care if a subject_id have at least 1 ventil support
# ventil = pd.read_csv("C:/Users/user/OneDrive - National University of Singapore/Desktop/MPH study documents/Y2S2/SPH5104/To join/ventilation.csv").drop_duplicates()
# ventil = pd.merge(left=ventil, right=ventil.groupby('subject_id')['ventilation_st'].sum().reset_index().rename(columns={'ventilation_st': 'ventil_st_sum'}), on='subject_id', how='left')
# ventil = ventil[['subject_id','hadm_id','stay_id', 'ventil_st_sum']].drop_duplicates()
# check_unique_subject_id(ventil3)

In [24]:
icu_label_admit_ptage_combine.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75


In [25]:
sofa.head()

,subject_id,hadm_id,stay_id,input,first_day_sofa
0,11058391,23179805,32195624,Enoxaparin,1.0
1,12298087,23902383,36364779,Enoxaparin,8.0
2,13905814,21346286,30922857,Enoxaparin,3.0
3,10495076,27394752,37292918,Enoxaparin,5.0
4,11111901,29789360,30265100,Enoxaparin,3.0


In [26]:
combine1 = pd.merge(left=icu_label_admit_ptage_combine, right=sofa, on=['subject_id', 'hadm_id', 'stay_id'], how='left')
check_unique_subject_id(combine1)

total_len: 7954, total_unique_id: 7954


In [27]:
charlson.head()

,subject_id,hadm_id,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index
0,11948471,24089481,4,1,0,0,0,1,0,0,0,0,1,0,0,8
1,10082986,27812341,4,0,0,0,0,0,0,0,1,0,0,0,0,5
2,10793648,22103020,3,0,0,0,0,0,0,0,0,0,0,0,0,3
3,16290929,25502564,1,0,0,0,0,0,0,0,1,0,1,0,0,4
4,13125781,25976500,2,0,0,0,0,0,0,0,0,0,1,0,1,8


In [28]:
combine2 =pd.merge(left= combine1, right=charlson, on=['subject_id', 'hadm_id'], how='left')
check_unique_subject_id(combine2)

total_len: 7954, total_unique_id: 7954


In [29]:
combine2.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55,Heparin,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82,Heparin,5.0,4,1,1,0,0,0,0,0,0,1,1,0,0,10
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81,Heparin,4.0,4,0,1,0,0,0,0,0,0,0,0,0,0,6
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73,Heparin,3.0,3,0,0,0,0,0,0,0,0,0,1,0,0,5
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75,Heparin,22.0,3,0,1,0,0,0,0,0,0,1,1,0,0,8


In [30]:
avgplatlets.head()

,subject_id,hadm_id,stay_id,input,avg_pl
0,10383338,28299477,37051184,Heparin,208.80
1,11018735,21052954,35153005,Heparin,178.88
2,12118241,24952894,35497370,Heparin,177.63
3,12944046,20410940,36329590,Heparin,308.03
4,13030541,29372534,33014693,Heparin,91.69


In [31]:
combine3 =pd.merge(left= combine2, right=avgplatlets, on=['subject_id', 'hadm_id', 'stay_id', 'input'], how='left')
check_unique_subject_id(combine3)

total_len: 7954, total_unique_id: 7954


In [32]:
combine3.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index,avg_pl
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55,Heparin,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,349.33
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82,Heparin,5.0,4,1,1,0,0,0,0,0,0,1,1,0,0,10,271.80
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81,Heparin,4.0,4,0,1,0,0,0,0,0,0,0,0,0,0,6,217.15
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73,Heparin,3.0,3,0,0,0,0,0,0,0,0,0,1,0,0,5,209.62
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75,Heparin,22.0,3,0,1,0,0,0,0,0,0,1,1,0,0,8,79.55


In [33]:
#just to double check if it successfully match inputs when joining.
set(combine3["input"])

{'Enoxaparin', 'Heparin'}

In [34]:
combine3.columns

Index(['subject_id', 'hadm_id', 'stay_id', 'first_careunit', 'last_careunit',
       'intime', 'outtime', 'los', 'label', 'count', 'admittime', 'dischtime',
       'deathtime', 'admission_type', 'admit_provider_id',
       'admission_location', 'discharge_location', 'insurance', 'language',
       'marital_status', 'race', 'edregtime', 'edouttime',
       'hospital_expire_flag', 'gender', 'anchor_age', 'anchor_year',
       'anchor_year_group', 'dod', 'real_age', 'input', 'first_day_sofa',
       'age_score', 'myocardial_infarct', 'congestive_heart_failure',
       'peripheral_vascular_disease', 'cerebrovascular_disease',
       'chronic_pulmonary_disease', 'peptic_ulcer_disease', 'diabetes_with_cc',
       'diabetes_without_cc', 'renal_disease', 'malignant_cancer',
       'severe_liver_disease', 'metastatic_solid_tumor',
       'charlson_comorbidity_index', 'avg_pl'],
      dtype='object')

In [35]:
age_eth_wt.head()

,subject_id,hadm_id,stay_id,input,hadm_id_1,age,hadm_id_2,race,deathtime,subject_id_1,stay_id_1,weight_admit,weight,weight_min,weight_max
0,10754405,25853748,37549824,Heparin,25853748,94,25853748,ASIAN - SOUTH EAST ASIAN,2168-03-04 03:12:00,10754405,37549824,54.0,54.0,54.0,54.0
1,11044346,29213329,35139839,Heparin,29213329,21,29213329,WHITE,NaN,11044346,35139839,70.0,70.0,70.0,70.0
2,18052788,20268735,39348717,Heparin,20268735,93,20268735,WHITE,NaN,18052788,39348717,54.8,54.8,54.8,54.8
3,10770900,29162633,33775723,Heparin,29162633,22,29162633,WHITE,NaN,10770900,33775723,63.6,63.6,63.6,63.6
4,14382318,22871773,37297001,Heparin,22871773,93,22871773,WHITE,NaN,14382318,37297001,63.0,63.0,63.0,63.0


In [36]:
#dropping repeated columns
age_eth_wt =age_eth_wt.drop(columns=['subject_id_1', 'stay_id_1', 'hadm_id_1', 'hadm_id_2', 'deathtime','race'])

In [37]:
age_eth_wt.head()

,subject_id,hadm_id,stay_id,input,age,weight_admit,weight,weight_min,weight_max
0,10754405,25853748,37549824,Heparin,94,54.0,54.0,54.0,54.0
1,11044346,29213329,35139839,Heparin,21,70.0,70.0,70.0,70.0
2,18052788,20268735,39348717,Heparin,93,54.8,54.8,54.8,54.8
3,10770900,29162633,33775723,Heparin,22,63.6,63.6,63.6,63.6
4,14382318,22871773,37297001,Heparin,93,63.0,63.0,63.0,63.0


In [38]:
combine4 =pd.merge(left= combine3, right=age_eth_wt, on=['subject_id', 'hadm_id', 'stay_id', 'input'], how='left')
check_unique_subject_id(combine4)

total_len: 7954, total_unique_id: 7954


In [39]:
combine4.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index,avg_pl,age,weight_admit,weight,weight_min,weight_max
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55,Heparin,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,349.33,55,71.2,71.200000,71.2,71.2
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82,Heparin,5.0,4,1,1,0,0,0,0,0,0,1,1,0,0,10,271.80,82,48.0,48.000000,48.0,48.0
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81,Heparin,4.0,4,0,1,0,0,0,0,0,0,0,0,0,0,6,217.15,81,48.4,48.966667,48.4,50.1
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73,Heparin,3.0,3,0,0,0,0,0,0,0,0,0,1,0,0,5,209.62,73,68.0,68.000000,68.0,68.0
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75,Heparin,22.0,3,0,1,0,0,0,0,0,0,1,1,0,0,8,79.55,75,93.0,94.133333,93.0,96.4


In [40]:
creatinine.head()

,Unnamed: 0,is_dup,subject_id,hadm_id,stay_id,amount,amountuom,first_careunit,age,input,avgw,gender,avg_creat,creat_score
0,0,False,16771333,23930162,32792750,1.0,dose,Medical/Surgical Intensive Care Unit (MICU/SICU),93,Enoxaparin,57.15,F,0.45,82.902778
1,1,False,12451629,29664742,32917422,1.0,dose,Medical/Surgical Intensive Care Unit (MICU/SICU),29,Enoxaparin,67.60,F,0.80,130.270833
2,2,False,12474402,27211588,35686350,1.0,dose,Medical/Surgical Intensive Care Unit (MICU/SICU),93,Heparin,73.05,F,1.25,38.148333
3,3,False,13241710,22990223,37121997,1.0,dose,Surgical Intensive Care Unit (SICU),20,Heparin,56.70,F,0.78,121.153846
4,4,False,15697584,29200059,37913496,1.0,dose,Medical/Surgical Intensive Care Unit (MICU/SICU),21,Heparin,82.20,M,0.90,150.953704


In [41]:
#dropping columns which we do not need
creatinine =creatinine.drop(columns=['Unnamed: 0', 'is_dup', 'amount', 'amountuom', 'first_careunit','age', 'gender'])

In [42]:
combine5 =pd.merge(left= combine4, right=creatinine, on=['subject_id', 'hadm_id', 'stay_id', 'input'], how='left')
check_unique_subject_id(combine5)

total_len: 7954, total_unique_id: 7954


In [43]:
combine5.head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index,avg_pl,age,weight_admit,weight,weight_min,weight_max,avgw,avg_creat,creat_score
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55,Heparin,0.0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,349.33,55,71.2,71.200000,71.2,71.2,71.20,0.57,147.465887
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82,Heparin,5.0,4,1,1,0,0,0,0,0,0,1,1,0,0,10,271.80,82,48.0,48.000000,48.0,48.0,48.00,2.38,16.246499
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81,Heparin,4.0,4,0,1,0,0,0,0,0,0,0,0,0,0,6,217.15,81,48.4,48.966667,48.4,50.1,50.45,0.33,125.275673
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73,Heparin,3.0,3,0,0,0,0,0,0,0,0,0,1,0,0,5,209.62,73,68.0,68.000000,68.0,68.0,68.00,1.18,53.625235
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75,Heparin,22.0,3,0,1,0,0,0,0,0,0,1,1,0,0,8,79.55,75,93.0,94.133333,93.0,96.4,95.91,0.80,108.231771


In [44]:
mortality.head()

,subject_id,hadm_id,stay_id,input,subject_id_y,discharge_location,died
0,18769980,23615370,36530181,Heparin,18769980,HOME HEALTH CARE,0
1,14124344,20533997,33346789,Heparin,14124344,CHRONIC/LONG TERM ACUTE CARE,0
2,11034601,23113363,38441444,Heparin,11034601,HOME HEALTH CARE,0
3,13560848,29641341,35627635,Heparin,13560848,SKILLED NURSING FACILITY,0
4,17792682,23446500,36740475,Heparin,17792682,REHAB,0


In [45]:
mortality =mortality.drop(columns=['subject_id_y'])

In [46]:
combine6 =pd.merge(left= combine5, right=mortality, on=['subject_id', 'hadm_id', 'stay_id', 'input'], how='left')
check_unique_subject_id(combine6)

total_len: 7954, total_unique_id: 7954


In [47]:
vasopr.head()

,subject_id,hadm_id,stay_id,input,if_antithrombotic_agents,if_vasopressin
0,18477317,25174049,39371812,Enoxaparin,0,0
1,13634631,25020057,31069020,Enoxaparin,0,0
2,17214442,29932944,33662100,Enoxaparin,0,0
3,16326529,26064171,37851970,Enoxaparin,0,0
4,15418457,27161339,32868979,Enoxaparin,0,0


In [48]:
combine7 =pd.merge(left= combine6, right=vasopr, on=['subject_id', 'hadm_id', 'stay_id', 'input'], how='left')
check_unique_subject_id(combine7)

total_len: 7954, total_unique_id: 7954


In [49]:
ventil.head()

,subject_id,hadm_id,stay_id,have_at_least_1_ventil
0,10522319,20271144,31949897,1
2,10630310,29121601,36606626,1
3,10833304,26184887,38948190,0
4,11008656,29884946,36566693,1
8,11108531,22258773,34419965,0


Ventilation status coded as:
none = 0
non-invasive = 1
invasive = 2

In [50]:
combine8 =pd.merge(left= combine7, right=ventil, on=['subject_id', 'hadm_id', 'stay_id'], how='left')
check_unique_subject_id(combine8)

total_len: 7954, total_unique_id: 7954


In [51]:
icd_dx_combine.head()

,icd_code,icd_version,long_title,subject_id,hadm_id,seq_num
3,0020,9,Typhoid fever,10158684,22072764,1.0
4,0020,9,Typhoid fever,10427677,23641430,1.0
5,0020,9,Typhoid fever,11088311,27128981,1.0
6,0020,9,Typhoid fever,11816842,25448380,1.0
7,0020,9,Typhoid fever,12916672,24528914,1.0


In [52]:
#### this is to search for subject_id and hadam_id pair that have vte or not
combine8 = combine8.reset_index(drop=True)
combine8['have_vte'] = 0
for idx, row in combine8.iterrows():
    if(idx%500==0):
        print(idx)
    curr_si = row['subject_id']
    curr_hi =  row['hadm_id']
    curr_icd_dx = icd_dx_combine[(icd_dx_combine['subject_id']==curr_si) & (icd_dx_combine['hadm_id']==curr_hi)].reset_index(drop=True)
    if (len(curr_icd_dx) ==0):
        have_vte = 0
        combine8.loc[idx, 'have_vte'] = have_vte
        continue
    have_vte = 0
    curr_icd_dx['long_title'] = curr_icd_dx['long_title'].str.lower()
    for idx2, row2 in curr_icd_dx.iterrows():
        curr_lt = row2['long_title']
        if ((("thrombosis" in curr_lt) or ("embolism" in curr_lt) or ("thromboembolism" in curr_lt)) and not(('chronic' in curr_lt) or ('Chronic' in curr_lt)) ):
            have_vte = 1
            combine8.loc[idx, 'have_vte'] = have_vte
            break            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [53]:
# to check for vte per group
for i in set(combine8['label']):
    print(i)
    print(f"have_vte: {len(combine8[(combine8['have_vte']==1) & (combine8['label']==i)])}, no_vte: {len(combine8[(combine8['have_vte']==0) & (combine8['label']==i)])}")

Heparin Sodium (Prophylaxis)
have_vte: 915, no_vte: 6723
Enoxaparin (Lovenox)
have_vte: 148, no_vte: 168


In [55]:
#set(icd_dx_combine[icd_dx_combine['long_title'].str.contains("intracerebral hemorrhage")]["long_title"])
allbleedinghits_initial =[
    'Nontraumatic intracerebral hemorrhage in brain stem',
 'Nontraumatic intracerebral hemorrhage in cerebellum',
 'Nontraumatic intracerebral hemorrhage in hemisphere, cortical',
 'Nontraumatic intracerebral hemorrhage in hemisphere, subcortical',
 'Nontraumatic intracerebral hemorrhage in hemisphere, unspecified',
 'Nontraumatic intracerebral hemorrhage, intraventricular',
 'Nontraumatic intracerebral hemorrhage, multiple localized',
 'Nontraumatic intracerebral hemorrhage, unspecified',
 'Other nontraumatic intracerebral hemorrhage', 'Nontraumatic subarachnoid hemorrhage from left anterior communicating artery',
 'Nontraumatic subarachnoid hemorrhage from left carotid siphon and bifurcation',
 'Nontraumatic subarachnoid hemorrhage from left middle cerebral artery',
 'Nontraumatic subarachnoid hemorrhage from left posterior communicating artery',
 'Nontraumatic subarachnoid hemorrhage from left vertebral artery',
 'Nontraumatic subarachnoid hemorrhage from other intracranial arteries',
 'Nontraumatic subarachnoid hemorrhage from right anterior communicating artery',
 'Nontraumatic subarachnoid hemorrhage from right carotid siphon and bifurcation',
 'Nontraumatic subarachnoid hemorrhage from right middle cerebral artery',
 'Nontraumatic subarachnoid hemorrhage from right posterior communicating artery',
 'Nontraumatic subarachnoid hemorrhage from right vertebral artery',
 'Nontraumatic subarachnoid hemorrhage from unspecified anterior communicating artery',
 'Nontraumatic subarachnoid hemorrhage from unspecified intracranial artery',
 'Nontraumatic subarachnoid hemorrhage from unspecified middle cerebral artery',
 'Nontraumatic subarachnoid hemorrhage, unspecified',
 'Other nontraumatic subarachnoid hemorrhage',
 'Gastrointestinal hemorrhage, unspecified',
 'Esophageal varices with bleeding',
'Acute duodenal ulcer with hemorrhage',
'Acute peptic ulcer with hemorrhage',
'Acute gastrojejunal ulcer with hemorrhage',
'Acute gastrojejunal ulcer with both hemorrhage and perforation',
'Acute gastric ulcer with hemorrhage, without mention of obstruction',
'Acute gastric ulcer with hemorrhage, with obstruction',
'Acute gastric ulcer with hemorrhage and perforation, without mention of obstruction',
'Acute gastric ulcer with hemorrhage and perforation, with obstruction',
'Acute gastric ulcer with hemorrhage',
'Hemorrhage from respiratory passages', 
'Hemorrhage from other sites in respiratory passages', 
'Hemorrhage, not elsewhere classified', 
'Hemopericardium'
]

allbleedinghits = [x.lower() for x in allbleedinghits_initial]



In [56]:
#### this is to search for subject_id and hadam_id pair that have heparin-induced thrombocytopenia or not (yes ==1, no ==0)
combine8 = combine8.reset_index(drop=True)
combine8['HIT'] = 0
for idx, row in combine8.iterrows():
    if(idx%500==0):
        print(idx)
    curr_si = row['subject_id']
    curr_hi =  row['hadm_id']
    curr_icd_dx = icd_dx_combine[(icd_dx_combine['subject_id']==curr_si) & (icd_dx_combine['hadm_id']==curr_hi)].reset_index(drop=True)
    if (len(curr_icd_dx) ==0):
        HIT = 0
        combine8.loc[idx, 'HIT'] = HIT
        continue
    HIT = 0
    curr_icd_dx['long_title'] = curr_icd_dx['long_title'].str.lower()
    for idx2, row2 in curr_icd_dx.iterrows():
        curr_lt = row2['long_title']
        if (("heparin induced thrombocytopenia (hit)" in curr_lt) or ("heparin-induced thrombocytopenia (hit)" in curr_lt)):
            HIT = 1
            combine8.loc[idx, 'HIT'] = HIT
            break            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [57]:
#### this is to search for subject_id and hadam_id pair that have major bleeding or not (yes ==1, no ==0)
combine8 = combine8.reset_index(drop=True)
combine8['major_bleeding'] = 0
for idx, row in combine8.iterrows():
    if(idx%500==0):
        print(idx)
    curr_si = row['subject_id']
    curr_hi =  row['hadm_id']
    curr_icd_dx = icd_dx_combine[(icd_dx_combine['subject_id']==curr_si) & (icd_dx_combine['hadm_id']==curr_hi)].reset_index(drop=True)
    if (len(curr_icd_dx) ==0):
        major_bleeding = 0
        combine8.loc[idx, 'major_bleeding'] = major_bleeding
        continue
    major_bleeding = 0
    curr_icd_dx['long_title'] = curr_icd_dx['long_title'].str.lower()
    for idx2, row2 in curr_icd_dx.iterrows():
        curr_lt = row2['long_title']
        if (curr_lt in allbleedinghits):
            major_bleeding = 1
            combine8.loc[idx, 'major_bleeding'] = major_bleeding
            break            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [58]:
#### this is to search for subject_id and hadam_id pair that have sepsis events or not (yes ==1, no ==0)
combine8 = combine8.reset_index(drop=True)
combine8['have_sepsis'] = 0
for idx, row in combine8.iterrows():
    if(idx%500==0):
        print(idx)
    curr_si = row['subject_id']
    curr_hi =  row['hadm_id']
    curr_icd_dx = icd_dx_combine[(icd_dx_combine['subject_id']==curr_si) & (icd_dx_combine['hadm_id']==curr_hi)].reset_index(drop=True)
    if (len(curr_icd_dx) ==0):
        have_sepsis = 0
        combine8.loc[idx, 'have_sepsis'] = have_sepsis
        continue
    have_sepsis = 0
    curr_icd_dx['long_title'] = curr_icd_dx['long_title'].str.lower()
    for idx2, row2 in curr_icd_dx.iterrows():
        curr_lt = row2['long_title']
        if (("Sepsis" in curr_lt) or ("sepsis" in curr_lt)):
            have_sepsis = 1
            combine8.loc[idx, 'have_sepsis'] = have_sepsis
            break            

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500


In [59]:
#random check
icd_dx_combine[(icd_dx_combine['subject_id']==10215159) & (icd_dx_combine['hadm_id']==24039782)]
combine8[combine8['major_bleeding']==1].head(1)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location_x,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index,avg_pl,age,weight_admit,weight,weight_min,weight_max,avgw,avg_creat,creat_score,discharge_location_y,died,if_antithrombotic_agents,if_vasopressin,have_at_least_1_ventil,have_vte,HIT,major_bleeding,have_sepsis
163,10215159,24039782,34755606,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2127-12-14 20:04:00,2127-12-20 21:47:17,6.071725,Heparin Sodium (Prophylaxis),1.0,2127-12-14 19:13:00,2128-01-15 17:30:00,NaN,EW EMER.,P47SIK,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,DIVORCED,WHITE,2127-12-14 17:20:00,2127-12-14 20:04:00,0,F,67,2124,2011 - 2013,NaN,70,Heparin,7.0,2,0,0,0,1,1,0,0,1,0,0,0,0,5,356.76,70,73.0,72.2,71.4,73.0,68.53,0.75,88.835185,SKILLED NURSING FACILITY,0,1,0,1,0,0,1,0


In [60]:
# to encode our group
combine9 = combine8.copy()
combine9['encoded_input'] = -1
combine9.loc[combine9['label']== 'Heparin Sodium (Prophylaxis)', 'encoded_input'] = 1
combine9.loc[combine9['label']== 'Enoxaparin (Lovenox)', 'encoded_input'] = 0

In [61]:
combine9[(combine9['input']=='Enoxaparin') & (combine8['have_vte']==1)].head()

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location_x,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,diabetes_with_cc,diabetes_without_cc,renal_disease,malignant_cancer,severe_liver_disease,metastatic_solid_tumor,charlson_comorbidity_index,avg_pl,age,weight_admit,weight,weight_min,weight_max,avgw,avg_creat,creat_score,discharge_location_y,died,if_antithrombotic_agents,if_vasopressin,have_at_least_1_ventil,have_vte,HIT,major_bleeding,have_sepsis,encoded_input
52,10071869,20895291,34235968,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2110-12-31 23:22:00,2111-01-12 01:28:43,11.087998,Enoxaparin (Lovenox),1.0,2110-12-31 21:17:00,2111-01-12 00:15:00,2111-01-12 00:15:00,EW EMER.,P3529J,EMERGENCY ROOM,DIED,Medicare,ENGLISH,MARRIED,WHITE,2110-12-31 19:44:00,2110-12-31 23:22:00,1,M,85,2110,2014 - 2016,2111-01-12,85,Enoxaparin,5.0,4,0,1,0,0,0,0,0,0,0,0,0,0,7,152.08,85,79.0,79.00,79.0,79.0,74.51,0.77,73.918651,DIED,1,1,0,1,1,0,0,0,0
80,10110363,21842992,38812969,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2167-06-14 19:06:00,2167-06-17 22:39:11,3.148044,Enoxaparin (Lovenox),1.0,2167-06-14 17:53:00,2167-06-22 15:43:00,NaN,EW EMER.,P586YL,EMERGENCY ROOM,SKILLED NURSING FACILITY,Medicare,ENGLISH,MARRIED,WHITE,2167-06-14 13:51:00,2167-06-14 19:06:00,0,M,77,2167,2011 - 2013,2167-07-19,77,Enoxaparin,2.0,3,0,0,0,1,0,0,0,0,0,1,0,1,10,162.78,77,97.3,97.30,97.3,97.3,97.55,0.88,96.995739,SKILLED NURSING FACILITY,0,0,0,1,1,0,0,0,0
174,10224976,26148096,33880358,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2164-02-21 19:11:00,2164-02-26 23:55:32,5.197593,Enoxaparin (Lovenox),1.0,2164-02-21 17:33:00,2164-03-07 15:00:00,2164-03-07 15:00:00,EW EMER.,P7554I,EMERGENCY ROOM,DIED,Other,ENGLISH,MARRIED,WHITE,2164-02-21 14:23:00,2164-02-21 19:11:00,1,M,38,2163,2011 - 2013,2164-03-07,39,Enoxaparin,3.0,0,0,0,0,0,0,0,0,0,0,1,0,1,6,231.20,39,100.9,102.45,100.9,104.0,105.76,0.70,211.939683,DIED,1,0,0,1,1,0,0,0,0
185,10247357,24391454,31726457,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2124-06-29 05:42:51,2124-07-04 05:22:15,4.985694,Enoxaparin (Lovenox),1.0,2124-06-28 14:58:00,2124-07-04 00:30:00,2124-07-04 00:30:00,EW EMER.,P11CH5,EMERGENCY ROOM,DIED,Medicaid,?,MARRIED,ASIAN,2124-06-28 11:12:00,2124-06-28 17:13:00,1,M,67,2124,2008 - 2010,2124-07-04,67,Enoxaparin,4.0,2,0,0,0,0,0,0,0,0,0,1,0,1,8,157.00,67,52.0,52.00,52.0,52.0,52.00,1.16,45.450192,DIED,1,0,0,1,1,0,0,1,0
394,10495076,27394752,37292918,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2170-08-17 21:54:05,2170-08-22 18:49:26,4.871771,Enoxaparin (Lovenox),1.0,2170-08-17 00:33:00,2170-08-26 12:40:00,NaN,SURGICAL SAME DAY ADMISSION,P28OI9,PHYSICIAN REFERRAL,HOME,Medicare,ENGLISH,DIVORCED,WHITE,NaN,NaN,0,M,68,2170,2014 - 2016,NaN,68,Enoxaparin,5.0,2,0,0,0,0,0,0,0,0,0,1,0,0,4,101.72,68,65.0,65.00,65.0,65.0,70.97,0.61,116.344262,HOME,0,0,0,1,1,0,0,0,0


In [63]:
# randomcheck= icd_dx_combine[(icd_dx_combine['subject_id']==10071869) & (icd_dx_combine['hadm_id']==20895291)]
# combine9[(combine9['subject_id']==10071869) & (combine9['hadm_id']==20895291)]
check_unique_subject_id(combine9)

total_len: 7954, total_unique_id: 7954


"""
1) You have a dataset with ppl that have taken Enx or taken Hep
2) create a column called treated, for ppl that took Enx label as 1, for ppl that took Hep label 0 (up to you to decide which is 1 or 0)
3) Feed a logisitic regresion with relevant features and with label treated.
4) This model will output a probability score aka propensity score
5) Lets call the lesser number dataset A and the larger number dataset B,
            matching_data_B = []
            for idxa, rowa in A.itterows():
                row_propensity_score_A=model.predict(rowa[features])
                tempB = B.copy()
                tempB['prop_score_diff'] = np.inf
                for idxb, rowb in tempB.itterows():
                    row.loc[idxb, 'prop_score_diff'] = abs(model.predict(rowb[features]) - row_propensity_score_A)
                tempB=tempB.sort_values(by='prop_score_diff')
                matching_data_B.append(tempB.head(1))
6) finally you would have dataset A and matching_data_B that matches A on propensity score
"""

In [64]:
combine10 = combine9.copy()
combine10.head()
combine10['have_diabetes'] = 0
combine10.loc[(combine10['diabetes_with_cc']==1) | (combine10['diabetes_without_cc']==1), 'have_diabetes'] = 1
combine10['have_cancer'] = 0
combine10.loc[(combine10['malignant_cancer']==1) | (combine10['metastatic_solid_tumor']==1), 'have_cancer'] = 1
combine10['creat_clr'] = 0
combine10.loc[combine10['creat_score']>=30, 'creat_clr'] = 1
combine10 = combine10.drop(columns=['diabetes_with_cc', 'diabetes_without_cc', 'malignant_cancer', 'metastatic_solid_tumor'])
combine10.head(10)

,subject_id,hadm_id,stay_id,first_careunit,last_careunit,intime,outtime,los,label,count,admittime,dischtime,deathtime,admission_type,admit_provider_id,admission_location,discharge_location_x,insurance,language,marital_status,race,edregtime,edouttime,hospital_expire_flag,gender,anchor_age,anchor_year,anchor_year_group,dod,real_age,input,first_day_sofa,age_score,myocardial_infarct,congestive_heart_failure,peripheral_vascular_disease,cerebrovascular_disease,chronic_pulmonary_disease,peptic_ulcer_disease,renal_disease,severe_liver_disease,charlson_comorbidity_index,avg_pl,age,weight_admit,weight,weight_min,weight_max,avgw,avg_creat,creat_score,discharge_location_y,died,if_antithrombotic_agents,if_vasopressin,have_at_least_1_ventil,have_vte,HIT,major_bleeding,have_sepsis,encoded_input,have_diabetes,have_cancer,creat_clr
0,10001217,24597018,37067082,Surgical Intensive Care Unit (SICU),Surgical Intensive Care Unit (SICU),2157-11-20 19:18:02,2157-11-21 22:08:00,1.118032,Heparin Sodium (Prophylaxis),1.0,2157-11-18 22:56:00,2157-11-25 18:00:00,NaN,EW EMER.,P4645A,EMERGENCY ROOM,HOME HEALTH CARE,Other,?,MARRIED,WHITE,2157-11-18 17:38:00,2157-11-19 01:24:00,0,F,55,2157,2011 - 2013,NaN,55,Heparin,0.0,1,0,0,0,0,0,0,0,0,1,349.33,55,71.2,71.200000,71.2,71.2,71.20,0.57,147.465887,HOME HEALTH CARE,0,0,0,1,0,0,0,0,1,0,0,1
1,10002155,28994087,31090461,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2130-09-24 00:50:00,2130-09-27 22:13:41,3.891447,Heparin Sodium (Prophylaxis),1.0,2130-09-23 21:59:00,2130-09-29 18:55:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,HOME HEALTH CARE,Other,ENGLISH,MARRIED,WHITE,2130-09-23 19:59:00,2130-09-24 00:50:00,0,F,80,2128,2008 - 2010,2131-03-10,82,Heparin,5.0,4,1,1,0,0,0,0,1,0,10,271.80,82,48.0,48.000000,48.0,48.0,48.00,2.38,16.246499,HOME HEALTH CARE,0,0,0,1,0,0,0,0,1,0,1,0
2,10002428,23473524,35479615,Surgical Intensive Care Unit (SICU),Medical Intensive Care Unit (MICU),2156-05-11 14:49:34,2156-05-22 14:16:46,10.977222,Heparin Sodium (Prophylaxis),1.0,2156-05-11 14:49:00,2156-05-22 14:16:00,NaN,EW EMER.,P3529J,EMERGENCY ROOM,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,WIDOWED,WHITE,2156-05-11 11:29:00,2156-05-11 16:53:00,0,F,80,2155,2011 - 2013,NaN,81,Heparin,4.0,4,0,1,0,0,0,0,0,0,6,217.15,81,48.4,48.966667,48.4,50.1,50.45,0.33,125.275673,CHRONIC/LONG TERM ACUTE CARE,0,0,0,1,0,0,0,1,1,0,0,1
3,10003019,22774359,30676350,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2175-10-08 18:58:00,2175-10-09 11:59:16,0.709213,Heparin Sodium (Prophylaxis),1.0,2175-10-08 13:56:00,2175-10-17 17:30:00,NaN,EW EMER.,P039RH,EMERGENCY ROOM,HOME HEALTH CARE,Medicare,ENGLISH,MARRIED,WHITE,2175-10-08 10:10:00,2175-10-08 18:58:00,0,M,69,2171,2008 - 2010,NaN,73,Heparin,3.0,3,0,0,0,0,0,0,0,0,5,209.62,73,68.0,68.000000,68.0,68.0,68.00,1.18,53.625235,HOME HEALTH CARE,0,0,0,0,0,0,0,1,1,0,1,1
4,10003400,20214994,32128372,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2137-02-25 23:37:19,2137-03-10 21:29:36,12.911308,Heparin Sodium (Prophylaxis),1.0,2137-02-24 10:00:00,2137-03-19 15:45:00,NaN,URGENT,P60ZCO,TRANSFER FROM SKILLED NURSING FACILITY,CHRONIC/LONG TERM ACUTE CARE,Medicare,ENGLISH,MARRIED,BLACK/AFRICAN AMERICAN,NaN,NaN,0,F,72,2134,2011 - 2013,2137-09-02,75,Heparin,22.0,3,0,1,0,0,0,0,1,0,8,79.55,75,93.0,94.133333,93.0,96.4,95.91,0.80,108.231771,CHRONIC/LONG TERM ACUTE CARE,0,0,0,1,1,0,0,0,1,0,1,1
5,10004733,27411876,39635619,Medical/Surgical Intensive Care Unit (MICU/SICU),Medical/Surgical Intensive Care Unit (MICU/SICU),2174-12-04 11:28:24,2174-12-12 20:03:01,8.357373,Heparin Sodium (Prophylaxis),1.0,2174-12-04 11:28:00,2174-12-27 14:00:00,NaN,URGENT,P48KFD,TRANSFER FROM HOSPITAL,HOME HEALTH CARE,Medicaid,ENGLISH,SINGLE,UNKNOWN,NaN,NaN,0,M,51,2174,2014 - 2016,NaN,51,Heparin,3.0,1,0,0,0,1,0,0,1,0,6,217.38,51,112.5,111.650000,110.8,112.5,109.66,3.13,43.307330,HOME HEALTH CARE,0,1,0,1,0,0,0,0

In [65]:
combine10['creat_score'].describe()

count    7870.000000
mean      100.559373
std        69.072956
min         1.111111
25%        53.535441
50%        89.537302
75%       132.366929
max      1884.141204
Name: creat_score, dtype: float64

In [66]:
#save as csv file
combine10.to_csv("./finalwithvte.csv",index=False)